In [70]:
import math
import numpy as np
import pandas as pd

In [71]:
# Retrieve Neel's "clean" files
appa1 = pd.read_csv('neel_exports/appa_hr_crop_withCreated.csv')
appa2 = pd.read_csv('neel_exports/appa_hr_2_crop_withCreated.csv')
appa1_gt = pd.read_csv('neel_exports/appa1_gt_clean_withCO.csv')
appa2_gt = pd.read_csv('data/appa2_gt.csv')

appa1_gt = appa1_gt.drop(columns='Unnamed: 0')


In [72]:
appa1.Time = pd.to_datetime(appa1.Time)
appa2.Time = pd.to_datetime(appa2.Time)

In [73]:
def add_times(df):
    # Add periodical time features to the dataset
    
    # HOUR OF THE DAY
    df["sin_hour"] = df["Time"].dt.hour.apply(
        lambda x: math.sin(x) * 2 * (math.pi / 24)
    )
    df["cos_hour"] = df["Time"].dt.hour.apply(
        lambda x: math.cos(x) * 2 * (math.pi / 24)
    )

    # DAY OF THE WEEK
    df["sin_weekday"] = df["Time"].dt.weekday.apply(
        lambda x: math.sin(x) * 2 * (math.pi / 7)
    )
    df["cos_weekday"] = df["Time"].dt.weekday.apply(
        lambda x: math.cos(x) * 2 * (math.pi / 7)
    )

    # MONTH OF THE YEAR
    df["sin_month"] = df["Time"].dt.month.apply(
        lambda x: math.sin(x) * 2 * (math.pi / 12)
    )

    df["cos_month"] = df["Time"].dt.month.apply(
        lambda x: math.cos(x) * 2 * (math.pi / 12)
    )

    # DAY OF THE YEAR (ORDINAL DATE)
    df["sin_ordate"] = df["Time"].apply(
        lambda x: math.sin(x.toordinal()) * 2 * (math.pi / 366)
    )
    df["cos_ordate"] = df["Time"].apply(
        lambda x: math.cos(x.toordinal()) * 2 * (math.pi / 366)
    )

    # Add the year feature
    df["year"] = df["Time"].dt.year

In [74]:
add_times(appa1)
add_times(appa2)

In [75]:
def rename_columns(df):
    df.columns = df.columns.str.replace('TimeSinceCreated', 'Age')

    df.columns = df.columns.str.replace('heatR_1', '1_heatR')
    df.columns = df.columns.str.replace('heatV_1', '1_heatV')
    df.columns = df.columns.str.replace('heatR_2', '2_heatR')
    df.columns = df.columns.str.replace('heatV_2', '2_heatV')

In [76]:
rename_columns(appa1)
rename_columns(appa2)

In [77]:
appa1.to_csv('noam_exports/appa1.csv')
appa2.to_csv('noam_exports/appa2.csv')

appa1_gt.to_csv('noam_exports/appa1_gt.csv')